In [ ]:
import matplotlib.pyplot as plt
from hoda_dataset_helper import read_hoda
import numpy as np
import os
import sys
import tarfile
import random
from keras.utils import np_utils
from PIL import Image
from IPython.display import display
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import AveragePooling2D
from keras.layers import Flatten
from keras.models import Sequential
from keras.models import load_model
from keras.optimizers import Adam
from scipy import ndimage
from __future__ import print_function
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle
%matplotlib inline

In [ ]:
X_train, Y_train, X_test, Y_test, X_remaining, Y_remaining = read_hoda()
print (X_train.shape)
print (X_train.astype)
#num_list = X_train.tolist()

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
#print (num_list)
X_train /= 255.0
X_test /= 255.0

Reading hoda/data/Train 60000.cdb
Reading hoda/data/Test 20000.cdb
Reading hoda/data/RemainingSamples.cdb
(60000, 32, 32, 1)
<built-in method astype of numpy.ndarray object at 0x7f3c713218f0>


In [ ]:
def train_valid_spliter(X_train,Y_train) :
    X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=1)
    return X_train,Y_train,X_val,Y_val

In [ ]:
X_train,Y_train,X_val,Y_val = train_valid_spliter(X_train,Y_train)


In [ ]:
print('train data shape: {}'.format(X_train.shape))
print('train labels shape: {}'.format(Y_train.shape))
print('valid data shape : {}'.format(X_val.shape))
print('valid labels shape: {}'.format(Y_val.shape))


train data shape: (48000, 32, 32, 1)
train labels shape: (48000, 10)
valid data shape : (12000, 32, 32, 1)
valid labels shape: (12000, 10)


In [ ]:
input_shape = X_train[0].shape
print(input_shape)
def conv_model(input_shape) :
    
    model = Sequential()
    model.add(Conv2D(16, 5, padding='same', activation='relu', input_shape=input_shape))
    model.add(Conv2D(16, 5, padding='same', activation='relu'))
    model.add(Conv2D(16, 5, padding='same', activation='relu'))
    model.add(Conv2D(16, 5, padding='same', activation='relu'))
    model.add(Conv2D(16, 5, padding='same', activation='relu'))
    model.add(Conv2D(16, 5, padding='same', activation='relu'))
    model.add(Conv2D(16, 5, padding='same', activation='relu'))
    model.add(Conv2D(16, 5, padding='same', activation='relu'))
    model.add(MaxPooling2D())

    model.add(Conv2D(32, 3, padding='same', activation='relu'))
    model.add(Conv2D(32, 3, padding='same', activation='relu'))
    model.add(Conv2D(32, 3, padding='same', activation='relu'))
    model.add(Conv2D(32, 3, padding='same', activation='relu'))
    model.add(Conv2D(32, 3, padding='same', activation='relu'))
    model.add(Conv2D(32, 3, padding='same', activation='relu'))
    model.add(Conv2D(32, 3, padding='same', activation='relu'))
    model.add(Conv2D(32, 3, padding='same', activation='relu'))
    model.add(MaxPooling2D())
    model.add(Conv2D(64, 3, padding='same', activation='relu'))
    model.add(Conv2D(64, 3, padding='same', activation='relu'))
    model.add(Conv2D(64, 3, padding='same', activation='relu'))
    model.add(Conv2D(64, 3, padding='same', activation='relu'))
    model.add(Conv2D(64, 3, padding='same', activation='relu'))
    model.add(Conv2D(64, 3, padding='same', activation='relu'))
    model.add(Conv2D(64, 3, padding='same', activation='relu'))
    model.add(Conv2D(64, 3, padding='same', activation='relu'))
    model.add(AveragePooling2D())

    model.add(Flatten())

    model.add(Dense(1024, activation='relu'))
    model.add(Dense(1024, activation='relu'))
    model.add(Dense(10, activation='softmax'))

    model.compile(optimizer=Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

(32, 32, 1)


In [ ]:
def train_model(input_shape,X_train,Y_train,X_val,Y_val,batch_size=64,epochs=100) :
        model = conv_model(input_shape)
        model.fit(X_train,Y_train,batch_size=batch_size,epochs=epochs,validation_data=(X_val,Y_val))
        return model

In [ ]:
model = train_model((32,32,1),X_train,Y_train,
                    X_val,Y_val,64,5)

Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 1353s 28ms/step - loss: 0.1355 - accuracy: 0.9582 - val_loss: 0.0595 - val_accuracy: 0.9813
Epoch 2/5
48000/48000 [==============================] - 1331s 28ms/step - loss: 0.0511 - accuracy: 0.9843 - val_loss: 0.0395 - val_accuracy: 0.9887
Epoch 3/5
48000/48000 [==============================] - 1357s 28ms/step - loss: 0.0414 - accuracy: 0.9877 - val_loss: 0.0408 - val_accuracy: 0.9892
Epoch 4/5
48000/48000 [==============================] - 1338s 28ms/step - loss: 0.0330 - accuracy: 0.9905 - val_loss: 0.0421 - val_accuracy: 0.9898
Epoch 5/5
48000/48000 [==============================] - 1330s 28ms/step - loss: 0.0327 - accuracy: 0.9911 - val_loss: 0.0645 - val_accuracy: 0.9831


In [ ]:
scores = model.evaluate(X_test,Y_test)
print('Accuracy : %{}'.format(scores[1]*100))
print("Error : %{}".format(100 - scores[1]*100))


20000/20000 [==============================] - 141s 7ms/step
Accuracy : %97.10999727249146
Error : %2.890002727508545
